In [1]:
import string 
import pandas as pd
import nltk
from nltk.tokenize import wordpunct_tokenize,word_tokenize
import os
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

dataset = pd.read_csv(os.getcwd()+'\\Ex_Files_NLP_Python_ML_EssT\\Exercise Files\\Ch01\\01_03\\Start\\SMSSpamCollection.tsv', header=None, sep='\t')
dataset.columns = ['label', 'body_text']

#create new features containing the percentage of punctuation in text and the body length of text 

def count_punctuation_and_body_len(text):
    punct_count = [char for char in text if char in string.punctuation]
    body_len = len(text) - text.count(" ")
    punct_perc = round(len(punct_count)/body_len, 3)*100
    return body_len, punct_perc

dataset['body_len'] = dataset['body_text'].apply(lambda x: count_punctuation_and_body_len(x)[0])
dataset['punc%'] = dataset['body_text'].apply(lambda x: count_punctuation_and_body_len(x)[1])

sp_words = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
def clean_data(text):
    tokens = ''.join([char for char in text if char not in string.punctuation])
    tokens_punct = re.split('\s+', tokens)
    tokens_stop = [token.lower()  for token in tokens_punct if token not in sp_words]
    clean_text = [ps.stem(word) for word in tokens_stop]
    return clean_text


**Step 1: Split into train/test**

In [3]:
from sklearn.model_selection import train_test_split

X_train, x_test, y_train, y_test = train_test_split(dataset[['body_text', 'body_len', 'punc%']], dataset['label'], test_size=0.2)

**Step 2: Vectorize text**

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer=clean_data)
tfidf_vect_fit = tfidf_vect.fit(X_train['body_text'])

tfidf_vect_train = tfidf_vect_fit.transform(X_train['body_text'])
tfidf_vect_test = tfidf_vect_fit.transform(x_test['body_text'])

x_train_vect = pd.concat([X_train[['body_len', 'punc%']].reset_index(drop=True), pd.DataFrame(tfidf_vect_train.toarray())],
                          axis=1)
x_test_vect = pd.concat([x_test[['body_len', 'punc%']].reset_index(drop=True), pd.DataFrame(tfidf_vect_test.toarray())], axis=1)

**Step3: Final evaluation of models**

In [11]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time

In [15]:
#Random_forest
rf = RandomForestClassifier(n_estimators=150, max_depth=None, n_jobs=-1)
start = time.time()
rf_model = rf.fit(x_train_vect, y_train)
end = time.time()
fit_time = end-start

start_pred = time.time()
y_pred = rf_model.predict(x_test_vect)
end_pred = time.time()
pred_time = end_pred - start_pred

precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')

print ('fit_time: {} / Predict_time: {} /Precision: {} / Recall: {} / fscore: {} / Accuracy: {}'.format(round(fit_time,3),
                                                                                     round(pred_time,3),
                                                                                     round(precision, 3),
                                                                      round(recall, 3),
                                                                      round(fscore, 3),
                                                                      round((y_pred == y_test).sum()/len(y_pred), 3)))

fit_time: 8.883 / Predict_time: 0.291 /Precision: 1.0 / Recall: 0.796 / fscore: 0.886 / Accuracy: 0.973


In [16]:
#Random_forest
gb = GradientBoostingClassifier(n_estimators=150, max_depth=11)
start = time.time()
gb_model = gb.fit(x_train_vect, y_train)
end = time.time()
fit_time = end-start

start_pred = time.time()
y_pred = rf_model.predict(x_test_vect)
end_pred = time.time()
pred_time = end_pred - start_pred

precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')

print ('fit_time: {} / Predict_time: {} /Precision: {} / Recall: {} / fscore: {} / Accuracy: {}'.format(round(fit_time,3),
                                                                                     round(pred_time,3),
                                                                                     round(precision, 3),
                                                                      round(recall, 3),
                                                                      round(fscore, 3),
                                                                      round((y_pred == y_test).sum()/len(y_pred), 3)))

fit_time: 391.995 / Predict_time: 0.295 /Precision: 1.0 / Recall: 0.796 / fscore: 0.886 / Accuracy: 0.973


Further Evaluation:
    * Slice the testset in a variety of ways
    * Examine the text messages that are mis-classified

Results trade off - Consider business context:
    * Is predict time really important
    * Precision - spam filter
    * recall - antivirus